<a href="https://colab.research.google.com/github/Sayed-Hossein-Hosseini/Node_Classification_in_the_Amazon_Product_Graph/blob/master/Edge_Prediction_in_the_Amazon_Product_Graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Edge Prediction in the Amazon Product Graph**

## **Library**

In [1]:
!pip install torch torchvision torchaudio
!pip install torch-geometric ogb
!pip install matplotlib

In [2]:
import torch
import torch.nn as nn
from torch.serialization import add_safe_globals
from torch_geometric.data.data import Data
from torch_geometric.data.data import DataEdgeAttr
from torch_geometric.data import Data
from torch_geometric.transforms import ToUndirected, RandomLinkSplit
from ogb.nodeproppred import NodePropPredDataset
from torch_geometric.loader import NeighborLoader
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, SAGEConv
from torch_geometric.utils import train_test_split_edges, negative_sampling
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import random

In [3]:
add_safe_globals({'torch_geometric.data.data.Data': Data})
add_safe_globals({'torch_geometric.data.data.DataEdgeAttr': DataEdgeAttr})

## **Loading the ogbn-products Dataset**

In [4]:
dataset = NodePropPredDataset(name='ogbn-products')
split_idx = dataset.get_idx_split()

graph, labels = dataset[0]

print(graph)

Downloaded 1.38 GB: 100%|██████████| 1414/1414 [00:24<00:00, 58.71it/s]


Extracting dataset/products.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:01<00:00,  1.02s/it]


Saving...
{'edge_index': array([[      0,  152857,       0, ..., 2449028,   53324, 2449028],
       [ 152857,       0,   32104, ...,  162836, 2449028,   53324]]), 'edge_feat': None, 'node_feat': array([[ 0.03193326, -0.1958605 ,  0.0519961 , ...,  0.07669606,
        -0.3929545 , -0.06478424],
       [-0.02405794,  0.63032097,  1.0605699 , ..., -1.6874819 ,
         3.5866776 ,  0.818219  ],
       [ 0.33269015, -0.5585958 , -0.28860757, ..., -0.37157044,
         0.2520575 ,  0.04153213],
       ...,
       [ 0.10660695,  0.2654852 , -0.00567423, ...,  1.0867023 ,
         0.07590195, -1.1736895 ],
       [ 0.24968362, -0.25740346,  0.41230008, ...,  1.5465808 ,
         1.0309792 , -0.29657176],
       [ 0.7175324 , -0.23930131,  0.04430327, ..., -1.0132493 ,
        -0.41407427, -0.08227058]], dtype=float32), 'num_nodes': 2449029}


## **Preprocessing and Preparing Data Graphs**

In [5]:
edge_index = torch.tensor(graph['edge_index'], dtype=torch.long)
x = torch.tensor(graph['node_feat'], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)

print(data)

Data(x=[2449029, 100], edge_index=[2, 123718280])


## **Graph Undirectedness**

In [6]:
# Convert the graph to undirected (i.e. (i→j) + (j→i))
data = ToUndirected()(data)
print(data)

Data(x=[2449029, 100], edge_index=[2, 123718152])
